In [ ]:
!pip install nfl_data_py

In [2]:
import nfl_data_py as nfl
import pandas as pd

In [3]:
nflpbp_df = nfl.import_pbp_data([2023, 2023, 2024])

2023 done.
2023 done.
2024 done.
Downcasting floats.


In [4]:
def get_field_goal_data(df: pd.DataFrame, offense, defense: str) -> pd.DataFrame:
  """Returns a DataFrame of field goal stats for the given play-by-play data.

  Finds all games where a field goal was made in the 4th quater which put the
  field goal kicker's team either tied or ahead of the opponent's score and
  where the kicker's team ultimately won the game.

  Returns:
    A DataFrame with:
    - The kicker's name
    - The expected points added for the made field goal
    - The number of seconds remaining in the game
    - Kick distance
  """
  out_df = df.loc[
    # Filter events by field goals made ...
    (df['field_goal_result'] == 'made') &
    # in the 4th quarter ...
    (df['qtr'] == 4) &
    (df['posteam_type'] == offense) &
    # where the kicker's team is trailing or tied before the field goal ...
    (df['posteam_score'] <= df[f'total_{defense}_score']) &
    # and leading or tied after the field goal is made ...
    (df['posteam_score_post'] >= df[f'total_{defense}_score']) &
    # and the kicker's team ultimately wins the game
    (df[f'{offense}_score'] > df[f'{defense}_score'])
    ][['kicker_player_name', f'{offense}_team', 'epa', 'game_seconds_remaining', 'kick_distance']]
  out_df = out_df.rename(columns={f'{offense}_team': 'team'})
  return out_df

In [5]:
home_fgs_df = get_field_goal_data(nflpbp_df, 'home', 'away')
away_fgs_df = get_field_goal_data(nflpbp_df, 'away', 'home')
fgs_df = pd.concat([home_fgs_df, away_fgs_df]).reset_index(drop=True)

In [6]:
kicker_gb = fgs_df.groupby('kicker_player_name')
# The kicker's team
team_s = kicker_gb['team'].unique()
# Number of made field goals by kicker
num_fgs_s = kicker_gb.size()
num_fgs_s.name = 'num_fgs'
# Average win probability added for each made field goal
wpa_avg_s = kicker_gb['epa'].mean()
wpa_avg_s.name = 'avg_epa'
# Average seconds remaining in the game for each made field goal
secs_remain_s = kicker_gb['game_seconds_remaining'].mean()
secs_remain_s.name = 'avg_secs_remain'
# Average distance of field goal in yards
kick_dist_s = kicker_gb['kick_distance'].mean()
kick_dist_s.name = 'avg_kick_distance'
fg_rank_df = pd.concat([team_s, num_fgs_s, wpa_avg_s, secs_remain_s, kick_dist_s], axis=1)

In [8]:
fg_rank_df.loc[fg_rank_df['num_fgs'] > 1].sort_values(
    by=['num_fgs', 'avg_epa', 'avg_secs_remain', 'avg_kick_distance'],
    ascending=[False, False, True, False])

,team,num_fgs,avg_epa,avg_secs_remain,avg_kick_distance
kicker_player_name,,,,,
H.Butker,[KC],13,0.253168,339.153839,30.307692
D.Hopkins,[CLE],10,0.702502,109.000000,39.000000
W.Lutz,[DEN],8,1.115860,176.750000,44.000000
Y.Koo,[ATL],8,0.661239,17.750000,42.000000
M.Gay,[IND],6,1.425053,376.500000,46.500000
G.Zuerlein,[NYJ],6,0.602503,40.666668,39.666668
T.Bass,[BUF],6,0.557458,88.666664,36.500000
J.Bates,[DET],5,0.870834,70.199997,44.200001
C.Ryland,"[ARI, NE]",5,0.756920,23.400000,42.599998
